In [1]:
import math
import numpy as np

In [2]:
import gensim
from gensim.models import word2vec
model_path = "models/glove_model2.txt"
#model= word2vec.Word2Vec.load_word2vec_format(model_path,binary=False) #GloVe Model 
model = gensim.models.KeyedVectors.load_word2vec_format(model_path,binary=False)

C:\Users\Administrator\Anaconda2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
model.similarity("chest", "legs")

0.6870097141153734

In [4]:
def similarity(words1, words2):
    shape = np.array(model[words1[0]], dtype = 'float32').shape
    words1_vector = np.zeros(shape, dtype = 'float32')
    words2_vector = np.zeros(shape, dtype = 'float32')
    for word in words1:
        words1_vector = words1_vector + np.array(model[word], dtype = 'float32')
    for word in words2:
        words2_vector = words2_vector + np.array(model[word], dtype = 'float32')
        
    score = math.sqrt(abs(np.dot(words1_vector, words2_vector)) / np.linalg.norm(words1_vector) / np.linalg.norm(words2_vector))
    #print np.dot(words1_vector, words2_vector), '-', np.linalg.norm(word1_vector), '-', np.linalg.norm(word2_vector)
    #print np.linalg.norm(words1_vector), '-', np.linalg.norm(model['increase'] + model['temperature'])
    print "Result of comparison of ", words1, " and ", words2, " = ", score
    
filter = ['JJ', 'NN', 'NNS', 'PP', 'NP', 'ADJP', 'VBG', 'VBZ', 'S', 'VB']
    
def getTreeType(word):
    #if filter.index(word.label() >= 0):
    if (word.label() in filter):
        return True
    return False

def getType(word, index):
    if (word[index] in filter):
        return True
    return False

In [5]:
from nltk.parse.stanford import StanfordParser
import os

path_to_jar = "models/stanford-parser-full-2018-02-27/stanford-parser.jar"
path_to_models_jar = 'models/stanford-parser-full-2018-02-27/stanford-parser-3.9.1-models.jar'

os.environ['CLASSPATH'] = "/home/sda/extrapol/sch.nlp/stanford/stanford-parser-full-2018-02-27/*"

parser = StanfordParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)



C:\Users\Administrator\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPParser instead.
  if __name__ == '__main__':


In [6]:
from google.cloud import translate
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\workspace\google_creds.json"
print os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

translate_client = translate.Client()

def getTranslation(query_rus, on_print=False):
    translation = translate_client.translate(query_rus, target_language='en')
    return [translation['translatedText']]
    
print getTranslation("боль в животе")

C:\workspace\google_creds.json
[u'abdominal pain']


# Описание процесса перевода и разбора входящего запроса

In [7]:
from nltk import Tree

from googletrans import Translator

def getTreeRaw(query):
    result = parser.raw_parse(query)

    output = list(result)
    #print output
    subtrees = output[0].subtrees(filter = getTreeType)
    
    return subtrees


def getTree(query, on_print = False):
    result = parser.raw_parse(query)

    output = list(result)
    #print output
    subtrees = output[0].subtrees(filter = getTreeType)

    meanings = []

    maxMeaningLength = 0
    
    for tree in subtrees:

        if on_print:
            print tree
            
        poses = tree.pos()

        if on_print:
            print "All poses: ", poses

        item = []

        for pos in poses:
            if (getType(pos, 1)):
                item.append(pos[0])
                #print pos, "in filter"

        if (not(item in meanings))and(len(item) > 0):
            meanings.append(item)
            if (len(item) > maxMeaningLength):
                maxMeaningLength = len(item)
    
    return meanings

def getTranslationRus(query_eng):
    ts = Translator()
    
    query_rus = ts.translate(query_eng, src='en', dest= 'ru')
    
    query_rus = query_rus.extra_data['possible-translations'][0][2][0]
    
    return query_rus

"""
def getTranslation(query_rus, on_print = False):
    
    ts = Translator()

    #query_rus = 'раздражение в промежности'

    query_eng = ts.translate(query_rus, src='ru', dest= 'en')

    #print query_eng.extra_data['possible-translations'][0][2][0][0]
    
    result = []
    
    for t in query_eng.extra_data['possible-translations'][0][2]:
        result.append(t[0])
    
    #result.append(query_eng.text)
        
    if on_print:
        print "All possible translations: ", result
    
    return result
"""
def getTrees(querys):
    result = []
    maxLength = 0
    for t in querys:
        meanings = getTree(t)
        if (len(meanings)>maxLength):
            maxLength = len(meanings)
        
        maxLength = math.sqrt(maxLength)
        #print maxLength
        to_append = []
        for m in meanings:
            if (len(m) >= maxLength):
                to_append.append(m)
        
        result.append(to_append)
        
    return result

query = getTranslation('голова болит')
meanings_arr = getTrees(query)

print query
print meanings_arr

[u'my head hurts']
[[[u'head', u'hurts']]]


# Обновление словарей

In [8]:
import pickle

with open('sch.synonims.pickle', 'rb') as f:
    syn = pickle.load(f)
    
with open('sch.locuses.pickle', 'rb') as f:
    locuses = pickle.load(f)
    
with open('sch.features.pickle', 'rb') as f:
    features = pickle.load(f)
    
with open('sch.single.pickle', 'rb') as f:
    single = pickle.load(f)

In [9]:
dictionary = []

In [10]:
for s in syn:
    vector1 = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
    vector2 = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
    #print s
    for symp in s['symp']:
        symp = symp.replace('[', '')
        symp = symp.replace(']', '')
        symp = symp.replace('(', '')
        symp = symp.replace(')', '')
        try:
            vector1 = vector1 + model[symp]
        except:
            1+1
            #print "No such word in model ", symp
    
    hasLocus = False
    
    try:
        an = s['locus']
        hasLocus = True
    except:
        hasLocus = False
    
    if ((hasLocus)and(not(s['locus']==['body']))):
        for locus in s['locus']:
            try:
                vector2 = vector2 + model[locus]
            except:
                1 + 1
                print "No such word in model ", locus
                
    vector2 = vector2 + vector1
    
    if (s['n'] == 624):
        print s
    
    dictionary.append({
        'ori': s,
        'vec1': vector1,
        'vec2': vector2
    })

No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  OD
No such word in model  tarsum
No such word in model  tarsum
No such word in model  OD
No such word in model  OD
No such word in model  underlip
No such word in model  OD
No such word in model  OD
No such word in model  Left
No such word in model  Left
No such word in model  Left
No such word in model  Left
No such word in model  Left
No such word in model  Left
{'locus': [u'leather'], 'symp': [u'icterus'], 'n': 624}
{'locus': [u'leather'], 'symp': [u'yellowness', u'of', u'the', u'skin'], 'n': 624}
{'locus': [u'skin'], 'symp': [u'icterus'], 'n': 624}
{'locus': [u'skin'], 'symp': [u'yellowness', u'of', u'the', u'skin'], 'n': 624}
No such word in model  OD


# K-means indexing

In [11]:
from sklearn.cluster import KMeans

kdict = []

for i in range(len(dictionary)):
    kdict.append(dictionary[i]['vec1'])

In [12]:
n_clusters = 100

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(kdict)
#kmeans.labels_[0:10]

In [13]:
def countVectors(kmeans, X, n):
    counters = []
    for i in range(n):
        counters.append(0)
    
    predicted = kmeans.predict(X)
    
    for i in range(len(X)):
        counters[predicted[i]] += 1
        
    return counters

print countVectors(kmeans, kdict, n_clusters)

[132, 16, 50, 4, 2, 4, 23, 84, 35, 23, 60, 131, 425, 13, 42, 24, 12, 192, 4, 40, 4, 19, 106, 2, 11, 4, 125, 14, 20, 8, 97, 13, 99, 131, 2, 33, 32, 35, 19, 12, 33, 49, 9, 62, 10, 4, 4, 3, 23, 4, 22, 6, 6, 4, 14, 23, 6, 60, 16, 5, 65, 10, 46, 36, 4, 2, 4, 17, 6, 72, 12, 4, 48, 4, 6, 9, 63, 10, 10, 4, 72, 40, 2, 41, 28, 10, 27, 4, 42, 4, 2, 23, 24, 12, 4, 6, 71, 8, 4, 4]


In [14]:
def getIndexesOfCluster(kmeans, X, n):
    res = []
    
    predicted = kmeans.predict(X)
    
    for i in range(len(X)):
        if (predicted[i] == n):
            res.append(i)
    return res

inds = getIndexesOfCluster(kmeans, kdict, 0)


In [15]:
def printOrisFromIndexes(inds):
    for i in range(len(inds)):
        print dictionary[inds[i]]['ori']
        
printOrisFromIndexes(inds)

{'locus': [u'stomach'], 'symp': [u'ache'], 'n': 541}
{'locus': [u'belly'], 'symp': [u'ache'], 'n': 541}
{'locus': [u'head'], 'symp': [u'ache'], 'n': 556}
{'locus': [u'left', u'knee'], 'symp': [u'ache'], 'n': 569}
{'locus': [u'left', u'hip', u'joint'], 'symp': [u'ache'], 'n': 576}
{'locus': [u'left', u'hip'], 'symp': [u'ache'], 'n': 576}
{'locus': [u'left', u'wrist'], 'symp': [u'ache'], 'n': 578}
{'locus': [u'left', u'elbow', u'joint'], 'symp': [u'ache'], 'n': 584}
{'locus': [u'Left', u'elbow'], 'symp': [u'ache'], 'n': 584}
{'locus': [u'left', u'humerus'], 'symp': [u'ache'], 'n': 586}
{'locus': [u'Left', u'shoulder', u'joint'], 'symp': [u'ache'], 'n': 586}
{'locus': [u'right', u'knee'], 'symp': [u'ache'], 'n': 589}
{'locus': [u'right', u'hip', u'joint'], 'symp': [u'ache'], 'n': 596}
{'locus': [u'right', u'wrist'], 'symp': [u'ache'], 'n': 598}
{'locus': [u'right', u'elbow', u'joint'], 'symp': [u'ache'], 'n': 604}
{'locus': [u'right', u'elbow'], 'symp': [u'ache'], 'n': 604}
{'locus': [u'r

In [16]:
#центровые вектора кластеров
clusters = kmeans.cluster_centers_
cluster_index = []

for i in range(len(clusters)):
    cluster_index.append(getIndexesOfCluster(kmeans, kdict, i))

In [17]:
dicts = []
kdict_inds = kmeans.predict(kdict)

for i in range(n_clusters):
    dicts.append([])

for i in range(len(kdict_inds)):
    dicts[kdict_inds[i]].append(dictionary[i])

In [18]:
vec = model['fever']

def scalar(vec1, vec2):
    return math.sqrt(abs(np.dot(vec1, vec2)) / np.linalg.norm(vec1) / np.linalg.norm(vec2))

for i in range(len(clusters)):
    print i, " - ", scalar(vec, clusters[i])

0  -  0.625967100908
1  -  0.629587911817
2  -  0.613510886565
3  -  0.547595742535
4  -  0.549085558252
5  -  0.626466978532
6  -  0.605860640826
7  -  0.774741408583
8  -  0.643947681914
9  -  0.611014357382
10  -  0.459863782641
11  -  0.794778950428
12  -  0.5953933264
13  -  0.610072048009
14  -  0.704986397673
15  -  0.740776854554
16  -  0.624658216329
17  -  0.751403393178
18  -  0.561250518134
19  -  0.681813280522
20  -  0.637839880893
21  -  0.630315493601
22  -  0.624853823219
23  -  0.598171736971
24  -  0.639579144868
25  -  0.482243977762
26  -  0.558403601671
27  -  0.734401365654
28  -  0.586731290019
29  -  0.709244759947
30  -  0.634953293715
31  -  0.574064790462
32  -  0.714427630383
33  -  0.745568498121
34  -  0.581649992943
35  -  0.553006911562
36  -  0.654295183733
37  -  0.716678685965
38  -  0.635980430372
39  -  0.560088505997
40  -  0.658965565141
41  -  0.558171551457
42  -  0.592355368384
43  -  0.656865394075
44  -  0.621750159551
45  -  0.631068594729


# Вычисление векторов для словосочетаний

In [19]:
def getMeaningsVectorsArr(meanings_arr):
    meanings_vectors_arr = []
    for meanings in meanings_arr:
        meanings_vectors_arr.append(getMeaningsVectors(meanings))
    return meanings_vectors_arr

def getMeaningsVectors(meanings):

    meanings_vectors = []

    for mean in meanings:
        vector = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
        try:
            for m in mean:
                vector = vector + model[m]
            meanings_vectors.append({
                'ori': mean,
                'vec': vector
            })
        except:
            print "WARNING: no such words in model - ", mean
        
    return meanings_vectors

meanings_vectors_arr = getMeaningsVectorsArr(meanings_arr)
print meanings_vectors_arr

[[{'vec': array([-0.3745    ,  0.41782   ,  1.11695   , -1.7574999 , -0.11808997,
       -0.31575   , -0.849109  , -0.47305   ,  0.60493   , -0.34375998,
       -0.10629   ,  0.177939  ,  2.09102   ,  0.80323   ,  0.315449  ,
        0.5313    , -0.78131   , -0.09564   , -0.02174002,  0.7283701 ,
        0.44833   ,  0.75301   , -0.57382   ,  0.48157197,  0.525     ,
        1.1537    , -1.11568   , -1.6598899 ,  1.6319699 ,  0.341905  ,
        0.0311102 ,  1.2328501 ,  0.0508    , -0.141274  , -1.43834   ,
       -0.015826  , -0.65998   ,  1.07118   ,  0.521439  , -0.93412   ,
       -0.177923  , -0.90733004,  0.459578  , -0.17227   , -0.651245  ,
        0.31245   ,  0.68751   ,  0.51572   , -0.87829006, -1.83963   ,
       -0.448197  , -0.43819997, -0.69315803,  0.98525   , -0.551481  ,
       -3.0357502 , -0.15583   ,  0.14008999,  1.78636   ,  0.52759004,
        0.86717   ,  0.79054   ,  0.19492   ,  0.92337   , -0.66255   ,
        0.434742  ,  0.64986   ,  0.497294  ,  0.32474

# Функция вычисления score1 (по матрице сравнений)

In [20]:
import math

def getScore(arr1, arr2):
    m = np.zeros((len(arr1), len(arr2)), dtype = "float")
    for i in range(len(arr1)):
        for j in range(len(arr2)):
            try:
                m[(i, j)] = model.similarity(arr1[i], arr2[j])
            except:
                m[(i, j)] = 0.0
    
    score = []
    
    for i in range(min(len(arr1), len(arr2))):
        mx = np.max(m)
        n = np.argwhere(m == mx)[0]
        score.append(mx)
        m = np.delete(m, n[0], 0)
        m = np.delete(m, n[1], 1)
        
        
        #print m
        
    #print score
    
    result = 1
    
    for s in score:
        result = result * s
    
    if (result < 0):
        return 0
    
    result = math.pow(result, 1.0 / min(len(arr1), len(arr2)))
    
    return result
    
#getScore(["pain", "neck"],["head", "hurts"])

# Вычисление скора по модификаторам

In [21]:
modifiers = [
    {
        'name': 'color',
        'arr': ['color', 'yellow', 'blue', 'red', 'white', 'black', 'dark', 'yellowing', 'redness']
    },
    {
        'name': 'localization',
        'arr': ['right', 'left', 'upper', 'lower', 'up', 'low']
    },
    {
        'name': 'body_parts',
        'arr': ['hand', 'arm', 'leg', 'ankle', 'eye', 'cheek', 'foot']
    }
]

def getModifiers(ss):
    result = []
    for s in ss:
        class_found = False
        for i in range(len(modifiers)):
            mod = modifiers[i]
            for a in mod['arr']:
                s = s.lower()
                a = a.lower()
                try:
                    class_similarity = model.similarity(s, a)
                except:
                    class_similarity= 0
                if class_similarity > 0.8:
                    result.append({
                        's': s,
                        'class': mod['name'],
                        'class_n': i
                    })
                    class_found = True
                if class_found:
                    break;
            if (class_found):
                break;
    return result

def find(arr, field, value):
    for i in range(len(arr)):
        if arr[i][field] == value:
            return i
    return -1

MODIFIERS_PENALTY = 0.5

def compareModifiers(sym, query):
    result = 1.0
    for mq in query:
        index = find(sym, 'class_n', mq['class_n'])
        if (index >= 0):
                #print mq, sym[index]
                #result = result * model.similarity(mq['s'], sym[index]['s'])
                if (model.similarity(mq['s'], sym[index]['s']) < 0.66):
                    result = result * MODIFIERS_PENALTY
        else:
            result = result * MODIFIERS_PENALTY
        #print mq, result
    
    
    return result

def scoreModifiers(sym, query):
    sym_mod = getModifiers(sym)
    query_mod = getModifiers(query)
    #print sym_mod, query_mod
    return compareModifiers(sym_mod, query_mod)

#getModifiers(['yellow', 'right', 'hand'])

scoreModifiers([u'right', u'foot', u'pain'], [u'right', u'arm', u'hurts'])

0.5

# Вычисление всех скорингов

In [22]:
def getScoresArr(meanings_vectors_arr, all_dict = False):
    for i in range(len(meanings_vectors_arr)):
        meanings_vectors = getScores(meanings_vectors_arr[i], all_dict)
        
    return meanings_vectors_arr

def getDict_depr(mean):
    hs = 0
    hi = -1
    for i in range(len(clusters)):
        s = scalar(clusters[i], mean['vec'])
        if (s > hs):
            hs = s
            hi = i
            
    return hi

def getDict(mean):
    return kmeans.predict([mean['vec']])[0]

def getScores(meanings_vectors, all_dict = False):
    
    for mean in meanings_vectors:
        scores = []
        
        if (not(all_dict)):
            dict_ind = getDict(mean)
            if dict_ind > -1:
                custom_dict = dicts[dict_ind]
            else:
                custom_dict = []
                
            print "Custom dict-",dict_ind
        else:
            custom_dict = dictionary
            
        
        for s in custom_dict:
                        
            #score1 = np.dot(s['vec1'], mean['vec'])/np.linalg.norm(s['vec1'])/np.linalg.norm(mean['vec'])
            #score2 = np.dot(s['vec2'], mean['vec'])/np.linalg.norm(s['vec2'])/np.linalg.norm(mean['vec'])

            try:
                d = s['ori']['locus'] + s['ori']['symp']
            except:
                d = s['ori']['symp']
                #print "No locus", s['ori']
            m = mean['ori']
            
            #print m ,d 
            mod_score = scoreModifiers(d, m)

            score1 = getScore(m, d) * mod_score
            
            score2 = abs(np.dot(s['vec2'], mean['vec'])) /np.linalg.norm(s['vec2']) / np.linalg.norm(mean['vec']) * mod_score

            #print m,d, score1            
            if (math.isnan(score1)):
                score1 = 0
            if (math.isnan(score2)):
                score2 = 0

            try:
                scores.append((score1, s['ori']['n'], s['ori']['locus'], "score1"))
                scores.append((score2, s['ori']['n'], s['ori']['locus'], "score2"))
            except:
                scores.append((score1, s['ori']['n'], "", "score1"))
                scores.append((score2, s['ori']['n'], "", "score2"))
                
        
        scores_sorted = sorted(scores, key = lambda item: item[0])
        
        print "Printing new scores"
        for score in scores_sorted:
            if (score[1]==483):
                print score
        #mean['scores'] = scores_sorted[-10:]

        scores1_sorted = []
        scores1_indexes = []

        scores2_sorted = []
        scores2_indexes = []

        i = len(scores_sorted) - 1
        while ((len(scores1_sorted) < 5)and(i >= 0)):
            if (scores_sorted[i][3] == "score1"):
                isInList = False
                try:
                    if (scores1_indexes.index(scores_sorted[i][1]) > -1):
                        isInList = True
                except:
                    isInList = False
                if not(isInList):
                    scores1_sorted.append(scores_sorted[i])
                    scores1_indexes.append(scores_sorted[i][1])                #print i

            i = i - 1

        i = len(scores_sorted) - 1
        while ((len(scores2_sorted) < 5)and(i >= 0)):
            if (scores_sorted[i][3] == "score2"):
                isInList = False
                try:
                    if (scores2_indexes.index(scores_sorted[i][1]) > -1):
                        isInList = True
                except:
                    isInList = False
                if not(isInList):
                    scores2_sorted.append(scores_sorted[i])
                    scores2_indexes.append(scores_sorted[i][1])
                #print scores2_sorted, i

            i = i - 1

        mean['scores'] = scores1_sorted + scores2_sorted
        mean['vec'] = 0
        #print mean
    
    return meanings_vectors

#meanings_vectors_arr = getScoresArr(meanings_vectors_arr)


In [23]:
def getByNum(n):
    result = []
    for d in dictionary:
        if (d['ori']['n'] == n):
            return d 
        
def getAllByNum(n):
    result = []
    for d in dictionary:
        if (d['ori']['n'] == n):
            result.append(d)
    return result
        
def getMeanings(meanings_vectors_arr):
    result = []
    for meanings_vectors in meanings_vectors_arr:
        for meanings in meanings_vectors:
            for score in meanings['scores']:
                result.append((score[0], score[1], meanings_vectors[0]['ori'], getByNum(score[1])['ori'], score[3]))
    
    result = sorted(result, key = lambda item: item[4])
    
    scores1 = []
    scores2 = []
    
    i = 0
    while ((len(scores1) < 5)and(i < len(result))):
        if (result[i][4] == 'score1'):
            scores1.append(result[i])
            
        i = i + 1
            
    while ((len(scores2) < 5)and(i < len(result))):
        if (result[i][4] == 'score2'):
            scores2.append(result[i])
            
        i = i + 1
        
    result = scores1 + scores2
    
    result = sorted(result, key = lambda item: item[4])
    
    return result

def convertToJson(inp):
    result = []
    for i in range(len(inp)):
        t = inp[i]
        try:
            s = t[3]['locus'] + t[3]['symp']
        except:
            s = t[3]['symp']
        result.append({
            'score': t[0],
            'mean' : t[2],
            'n': t[1],
            'ori': {'s': s, 'n': t[3]['n']}
        })
    return result

result = getMeanings(meanings_vectors_arr)
print result

KeyError: 'scores'

In [24]:
for d in dictionary:
    if (d['ori']["n"]==483):
        print d

{'vec1': array([ 0.088963, -0.11943 ,  0.51479 , -0.22849 , -0.48744 ,  0.52461 ,
        1.0833  ,  0.2036  , -0.10356 , -0.90879 , -1.1133  , -0.046818,
        0.61796 ,  0.32025 ,  0.68944 ,  0.032507, -0.54395 , -0.6076  ,
        0.5394  , -0.96485 , -0.15272 ,  0.48759 , -0.54559 ,  0.26201 ,
        0.23242 ,  1.004   ,  0.46159 , -0.11139 ,  0.033382, -0.94896 ,
        0.35095 ,  0.37579 ,  0.4459  ,  0.21918 , -0.33345 ,  0.16264 ,
        0.17105 , -0.57323 , -0.21542 ,  0.28871 , -0.50265 ,  1.0135  ,
       -0.60055 , -1.8324  ,  1.0699  ,  0.30057 ,  0.044205, -0.36068 ,
        0.49848 , -0.25486 , -0.42697 ,  0.1646  , -0.35776 ,  0.33329 ,
        0.39519 , -1.523   ,  0.19788 ,  0.342   ,  0.42626 ,  0.83373 ,
        0.045401,  1.2959  , -0.17404 ,  0.023227, -0.43781 ,  1.1165  ,
        0.16485 , -0.6885  ,  0.64707 , -0.26326 , -0.90837 , -0.63386 ,
       -0.84953 ,  0.75681 ,  0.68023 ,  0.60484 , -0.19225 ,  0.22224 ,
       -0.94682 ,  0.080545,  0.19992 , -0

In [25]:
for i in range(100):
    d = dicts[i]
    for v in d:
        if (v['ori']['n']==483):
            print i

11
11
11
11
12
69
69
69
96


In [26]:
convertToJson(result)

NameError: name 'result' is not defined

In [27]:
def processQuery(query, all_dict = False):
    query = getTranslation(query)
    meanings_arr = getTrees(query)
    meanings_vectors_arr = getMeaningsVectorsArr(meanings_arr)
    meanings_vectors_arr = getScoresArr(meanings_vectors_arr, all_dict)
    result = getMeanings(meanings_vectors_arr)
    result = convertToJson(result)
    result = sorted(result, key = lambda item: -item['score'])
    return result

processQuery("давление")

Custom dict- 7
Printing new scores


[{'mean': [u'pressure'],
  'n': 1322,
  'ori': {'n': 1322,
   's': [u'abdomen',
    u'from',
    u'the',
    u'navel',
    u'to',
    u'the',
    u'pubic',
    u'region',
    u'pressure']},
  'score': 1.0},
 {'mean': [u'pressure'],
  'n': 780,
  'ori': {'n': 780,
   's': [u'fingers', u'of', u'the', u'right', u'hand', u'cold', u'snap']},
  'score': 0.7184235453605652},
 {'mean': [u'pressure'],
  'n': 1465,
  'ori': {'n': 1465, 's': [u'body', u'frequent', u'pulse']},
  'score': 0.7059236764907837},
 {'mean': [u'pressure'],
  'n': 1322,
  'ori': {'n': 1322,
   's': [u'abdomen',
    u'from',
    u'the',
    u'navel',
    u'to',
    u'the',
    u'pubic',
    u'region',
    u'pressure']},
  'score': 0.6996955871582031},
 {'mean': [u'pressure'],
  'n': 777,
  'ori': {'n': 777,
   's': [u'fingers', u'of', u'the', u'left', u'hand', u'cold', u'snap']},
  'score': 0.6993538737297058},
 {'mean': [u'pressure'],
  'n': 1401,
  'ori': {'n': 1401, 's': [u'body', u'instability', u'of', u'the', u'joints

In [28]:
import time

def accessPerf(query):
    t1 = time.time()
    result = processQuery(query, False)
    dt1 = time.time() - t1
    t2 = time.time()
    result = processQuery(query, True)
    dt2 = time.time() - t2
    print "For query - ", query, " - ", dt1, "secs", " gegen ", dt2, " secs"

accessPerf("боль в голове")
accessPerf("пелена перед глазами")
accessPerf("боль в животе")
accessPerf("болит живот")
accessPerf("болит голова")
accessPerf("повышенная температура")
accessPerf("головокружение")
accessPerf("боль в ноге")

Custom dict- 11
Printing new scores
(0.4575174152851105, 483, [u'the', u'body'], 'score2')
(0.6435675411666477, 483, [u'body'], 'score1')
(0.6435675411666477, 483, [u'the', u'body'], 'score1')
(0.6435675411666477, 483, '', 'score1')
(0.6435675411666477, 483, '', 'score1')
(0.6435675621032715, 483, [u'body'], 'score2')
(0.6435675621032715, 483, '', 'score2')
(0.6435675621032715, 483, '', 'score2')


C:\Users\Administrator\Anaconda2\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in float_scalars


Printing new scores
(0.2114520394864598, 483, '', 'score1')
(0.22825420424810866, 483, [u'body'], 'score1')
(0.22825420424810866, 483, [u'the', u'body'], 'score1')
(0.2324931025505066, 483, '', 'score2')
(0.23249310838487497, 483, '', 'score1')
(0.23633667826652527, 483, [u'body'], 'score2')
(0.23633667826652527, 483, '', 'score2')
(0.24706622958183289, 483, '', 'score2')
(0.2470662356187921, 483, '', 'score1')
(0.27153587341308594, 483, [u'the', u'body'], 'score2')
(0.4575174152851105, 483, [u'the', u'body'], 'score2')
(0.6435675411666477, 483, [u'body'], 'score1')
(0.6435675411666477, 483, [u'the', u'body'], 'score1')
(0.6435675411666477, 483, '', 'score1')
(0.6435675411666477, 483, '', 'score1')
(0.6435675621032715, 483, [u'body'], 'score2')
(0.6435675621032715, 483, '', 'score2')
(0.6435675621032715, 483, '', 'score2')
For query -  боль в голове  -  1.34999990463 secs  gegen  7.74300003052  secs
Custom dict- 80
Printing new scores
Printing new scores
(0.13030370271549127, 483, '', 

In [29]:
def checkTree(tree, ss):
    poses = tree.pos()
        
    for s in ss:
        found = True
        found_pos = False
        for pos in poses:
            if (pos[0] == s):
                found_pos = True
        if not(found_pos):
            found = False
            break
    return found            

def getSent(query, ss):
    
    query = getTranslation(query)[0]
    trees = getTreeRaw(query)
    
    appl_tree = ""
    appl_tree_len = 100
    
    for tree in trees: 
        if checkTree(tree, ss):
            if (len(tree.pos())< appl_tree_len):
                appl_tree = tree
                applt_tree_len = len(tree.pos())
                
    appl_tree = appl_tree.flatten()
    
    t = appl_tree.label()
    s = appl_tree.leaves()
    us = ""
    
    for i in range(len(s)):
        us += s[i] + " "
    
    #s = s.replace(t, "")
    #s = s.replace("(", "")
    #s = s.replace(")", "")
        
    query_rus = getTranslationRus(us)[0]
    query_rus = query_rus.lower()
    
    return t, query_rus
    

In [30]:
typ, sent = getSent('у меня болит голова', ["headache"])
print typ, sent

NN головная боль


In [31]:
processQuery('повышенная температура')

Custom dict- 11
Printing new scores
(0.6703671813011169, 483, [u'the', u'body'], 'score2')
(1.0000000000000002, 483, [u'body'], 'score1')
(1.0000000000000002, 483, [u'the', u'body'], 'score1')
(1.0000000000000002, 483, '', 'score1')
(1.0000000000000002, 483, '', 'score1')
(1.0000001192092896, 483, [u'body'], 'score2')
(1.0000001192092896, 483, '', 'score2')
(1.0000001192092896, 483, '', 'score2')


[{'mean': [u'fever'],
  'n': 483,
  'ori': {'n': 483, 's': [u'body', u'fever']},
  'score': 1.0000001192092896},
 {'mean': [u'fever'],
  'n': 483,
  'ori': {'n': 483, 's': [u'body', u'fever']},
  'score': 1.0000000000000002},
 {'mean': [u'fever'],
  'n': 683,
  'ori': {'n': 683, 's': [u'anus', u'diarrhea']},
  'score': 0.7167077631181513},
 {'mean': [u'fever'],
  'n': 527,
  'ori': {'n': 527, 's': [u'rib', u'cage', u'cough']},
  'score': 0.6714698672294617},
 {'mean': [u'fever'],
  'n': 540,
  'ori': {'n': 540, 's': [u'stomach', u'vomiting']},
  'score': 0.6605111956596375},
 {'mean': [u'fever'],
  'n': 527,
  'ori': {'n': 527, 's': [u'rib', u'cage', u'cough']},
  'score': 0.6584986660143968},
 {'mean': [u'fever'],
  'n': 540,
  'ori': {'n': 540, 's': [u'stomach', u'vomiting']},
  'score': 0.6581173753181658},
 {'mean': [u'fever'],
  'n': 539,
  'ori': {'n': 539, 's': [u'stomach', u'nausea']},
  'score': 0.6484075784683228},
 {'mean': [u'fever'],
  'n': 1068,
  'ori': {'n': 1068, 's': 

In [32]:
model.similarity('arm', 'foot')

0.5478494981867992

In [33]:
model.most_similar('jaundice')

[(u'vomiting', 0.6858890056610107),
 (u'nausea', 0.6830165386199951),
 (u'diarrhea', 0.6774415969848633),
 (u'rashes', 0.6471775770187378),
 (u'diarrhoea', 0.6440178155899048),
 (u'symptoms', 0.6343750357627869),
 (u'dysentery', 0.6342402696609497),
 (u'cirrhosis', 0.6254957318305969),
 (u'conjunctivitis', 0.620579719543457),
 (u'septicaemia', 0.6198016405105591)]

In [34]:
model['Red']

KeyError: "word 'Red' not in vocabulary"

In [ ]:
from flask import Flask, request, jsonify, Response, make_response

app = Flask(__name__)

@app.route('/scnlp/getsymptom', methods=['OPTIONS'])
def getSymptom_OPTIONS():
    #resp = Response(status=200)
    resp = make_response('200')
    resp.headers['Access-Control-Allow-Origin'] = '*'
    resp.headers['access-control-allow-origin'] = '*'
    resp.headers['access-control-allow-headers'] = 'access-control-allow-headers,access-control-allow-origin,content-type'
    resp.headers['content-type']="application/json"
    resp.headers['access-control-allow-methods']='GET,POST,OPTIONS'
    print "Options response", resp
    return resp

@app.route('/scnlp/getsymptom', methods=['GET', 'POST'])
def getSymptom():
    
    query = request.json['query']
    result = processQuery(query)
    print "Send back", result
    #resp = Response(response=jsonify({'result': result}))
    resp = make_response(jsonify({'result': result}));
    resp.headers['Access-Control-Allow-Origin'] = '*'
    #resp.headers['access-control-allow-origin'] = '*'
    resp.headers['access-control-allow-headers'] = 'access-control-allow-headers,access-control-allow-origin,content-type'
    resp.headers['content-type']="application/json"
    resp.headers['access-control-allow-methods']='GET,POST,OPTIONS'
    print "Send back", resp
    return resp

@app.route('/scnlp/sample', methods=['GET'])
def getSample():
    return jsonify({'result': processQuery("боль в груди")})

@app.route('/scnlp/check', methods=['GET'])
def check():
    return jsonify({'result': "Running OK"})

@app.route('/scnlp/sent', methods=['POST'])
def getSentAPI():
    query = request.json['query']
    ss = request.json['ss']
    t, s = getSent(query, ss)
    print t,s
    return jsonify({"type": t, "sent":s})

if __name__ == '__main__':
    #app.run(debug=True, port=8881, use_reloader=False)
    #app.run(debug=True, host="18.130.233.224", port=8881, use_reloader=False)
    app.run(debug=True, host="localhost", port=8881, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://localhost:8881/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2019 16:54:12] "OPTIONS /scnlp/getsymptom HTTP/1.1" 200 -


Options response <Response 3 bytes [200 OK]>
Custom dict- 17


127.0.0.1 - - [15/Mar/2019 16:54:14] "POST /scnlp/getsymptom HTTP/1.1" 200 -


Printing new scores
Send back [{'n': 1248, 'score': 1.0, 'ori': {'s': [u'neck', u'pain'], 'n': 1248}, 'mean': [u'pain']}, {'n': 667, 'score': 1.0, 'ori': {'s': [u'spine', u'pain'], 'n': 667}, 'mean': [u'pain']}, {'n': 556, 'score': 1.0, 'ori': {'s': [u'head', u'pain'], 'n': 556}, 'mean': [u'pain']}, {'n': 1713, 'score': 1.0, 'ori': {'s': [u'upper', u'eyelid', u'of', u'right', u'eye', u'pain'], 'n': 1713}, 'mean': [u'pain']}, {'n': 1711, 'score': 1.0, 'ori': {'s': [u'upper', u'eyelid', u'of', u'the', u'left', u'eye', u'pain'], 'n': 1711}, 'mean': [u'pain']}, {'n': 541, 'score': 0.9277157783508301, 'ori': {'s': [u'stomach', u'pain'], 'n': 541}, 'mean': [u'pain']}, {'n': 687, 'score': 0.9258033037185669, 'ori': {'s': [u'rib', u'cage', u'pain'], 'n': 687}, 'mean': [u'pain']}, {'n': 667, 'score': 0.9022372364997864, 'ori': {'s': [u'spine', u'pain'], 'n': 667}, 'mean': [u'pain']}, {'n': 1248, 'score': 0.8972241282463074, 'ori': {'s': [u'neck', u'pain'], 'n': 1248}, 'mean': [u'pain']}, {'n': 

...

Send back []
Send back <Response 19 bytes [200 OK]>


127.0.0.1 - - [11/Apr/2019 11:08:44] "OPTIONS /scnlp/getsymptom HTTP/1.0" 200 -


Options response <Response 3 bytes [200 OK]>


127.0.0.1 - - [11/Apr/2019 11:08:46] "POST /scnlp/getsymptom HTTP/1.0" 200 -


Custom dict- 69
Printing new scores
(0.1990999740710875, 483, '', 'score1')
(0.2641348499797784, 483, [u'body'], 'score1')
(0.2990705260079068, 483, [u'the', u'body'], 'score1')
(0.39371174573898315, 483, [u'body'], 'score2')
(0.39371174573898315, 483, '', 'score2')
(0.5135736465454102, 483, [u'the', u'body'], 'score2')
Send back [{'n': 933, 'score': 0.5936790704727173, 'ori': {'s': [u'body', u'gait', u'change'], 'n': 933}, 'mean': [u'\\', u'change']}, {'n': 1544, 'score': 0.5925167798995972, 'ori': {'s': [u'right', u'eye', u'obstruction', u'of', u'closing'], 'n': 1544}, 'mean': [u'\\', u'change']}, {'n': 1383, 'score': 0.5830599665641785, 'ori': {'s': [u'head', u'decreased', u'learning'], 'n': 1383}, 'mean': [u'\\', u'change']}, {'n': 1250, 'score': 0.5765465497970581, 'ori': {'s': [u'right', u'side', u'heaviness'], 'n': 1250}, 'mean': [u'\\', u'change']}, {'n': 1199, 'score': 0.5714797377586365, 'ori': {'s': [u'head', u'slowing', u'of', u'thinking'], 'n': 1199}, 'mean': [u'\\', u'cha